# Tutorial 2.1: Linearizing the Equations of Motion
In this tutorial you will learn to linearize the Equations of Motion of the pendulum as covered in the lecture. 

## Part 1: Kinematic equations

We first start by defining the variables. Let's start with the kinematic relations of a pendulum hanging from the point $(x_0, z_0)$.

In [1]:
from sympy import *

var("t x1 z1 r phi0")      
phi1 = Function("phi1")(t)
x2 = x1 + r*cos(phi1)
z2 = z1 + r*sin(phi1)

The velocities can then be obtained using:

In [2]:
xdot = diff(x2, t)
zdot = diff(z2, t)

## Part 2: Energy equations
### Kinetic energy:
The in this system is $m$, such that the kinetic energy is given by:

In [3]:
var("m")
T = 0.5*m*(xdot**2 + zdot**2)
T.evalf()

0.5*m*(r**2*sin(phi1(t))**2*Derivative(phi1(t), t)**2 + r**2*cos(phi1(t))**2*Derivative(phi1(t), t)**2)

This expression can be simplified to:

In [4]:
T = simplify(T)
T.evalf()

0.5*m*r**2*Derivative(phi1(t), t)**2

### Potential energy:
The potential energy on the pendulum is due to gravity:

In [5]:
var("g")
V = m*g*z2
V.evalf()

g*m*(r*sin(phi1(t)) + z1)

## Step 3: Construct the Lagrangian

In [6]:
L = T - V
L.evalf()

-g*m*(r*sin(phi1(t)) + z1) + 0.5*m*r**2*Derivative(phi1(t), t)**2

## Step 4: Obtaining the EoM

In order to obtain the EoMs we have to take derivatives w.r.t. $\phi_1$ and its velocity. 

In [7]:
EOM_phi = diff( diff(L, diff(phi1, t)), t) - diff(L, phi1)
EOM_phi.evalf()

g*m*r*cos(phi1(t)) + 1.0*m*r**2*Derivative(phi1(t), (t, 2))

The equation is nonlinear since it depends on the cosine of the angle. We want to obtain a linear function, so let's linearize. We consider a perturbation around the point we want to linearize ($\phi_0$). We linearize with the following function substitution: $\phi_1(t) = \phi_0 + \epsilon \psi(t)$. The $\epsilon$ is added in order to apply the Taylor series expansion. 

On a technical note: Using SymPy for function substitution can be tricky, hence we will use temporary symbols called `tmp1` and `tmp2` in order to do the substitution.

In [8]:
var("phi0 epsilon")
psi = Function("psi")(t)

tmp1, tmp2 = symbols("tmp1 tmp2")
EOM_phi = EOM_phi.evalf(subs={diff(phi1, (t, 2)): tmp2, phi1: tmp1})
EOM_phi = EOM_phi.evalf(subs={tmp2: diff(phi0 + epsilon*psi, (t, 2)), tmp1: phi0 + epsilon*psi})
EOM_phi.evalf()


1.0*epsilon*m*r**2*Derivative(psi(t), (t, 2)) + g*m*r*cos(epsilon*psi(t) + phi0)

Now, we can apply the Taylor series expansion using the function $\epsilon$ as a variable.

In [11]:
EOM_lin = series(EOM_phi, epsilon, n=2)
EOM_lin.evalf()

g*m*r*cos(phi0) + epsilon*(-g*m*r*psi(t)*sin(phi0) + 1.0*m*r**2*Derivative(psi(t), (t, 2))) + O(epsilon**2)

Then, we obtain the linearized EOM by setting $\epsilon = 1$.

In [18]:
EOM_lin = EOM_lin.removeO().evalf(subs={epsilon: 1})
EOM_lin.evalf()

-g*m*r*psi(t)*sin(phi0) + g*m*r*cos(phi0) + 1.0*m*r**2*Derivative(psi(t), (t, 2))

We see that we get an expression that only depends on (linearly) on the perturbation $\psi(t)$. Isolating the second derivative with respect to time of the perturbation, we get the final expression of the linearized system.

In [19]:
EOM_lin_iso = solve(EOM_lin, diff(psi, (t, 2)))
EOM_lin_iso[0].evalf()

g*(psi(t)*sin(phi0) - cos(phi0))/r

In this problem, our initial angle is $\phi_0 = 3*\frac{\pi}{2}$, then the final Equation of Motion will be:

In [21]:
EOM_lin_iso[0].evalf(subs={phi0: 3*pi/2})

-g*psi(t)/r

### Remarks

1) The problem studied here is already linear with respect to the accelerations and does not depend on velocities. If we had a nonlinear dependency on accelerations and/or velocities, we should have taken into account the gradients with respect these two quantities.
2) For a multiple DOF system, this process has to be done for all the EOM associated to each DOF.
